In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import SQL for the datafile
import sqlite3

# sklearn imports


### Importing SQL datafile

In [2]:
# create an SQL connection to read into pandas
con = sqlite3.connect('data/database.sqlite')

# pandas does not need a cursor object to read the table, but I make one to explore the data initially
cur = con.cursor()


In [3]:
cur.execute("""SELECT *
               FROM Team
               WHERE team_long_name == 'Liverpool'
""")
cur.fetchall()

[(3462, 8650, 9, 'Liverpool', 'LIV')]

In [4]:
# list of all tables names
table_names = ['Player_Attributes', 'Player', 'Match', 'League', 'Country', 'Team', 'Team_Attributes']

# loop through to query and extract individual tables
for t in table_names:
    # globals() allows us to use the string as a variable name
    # query pulls everything from each table
    globals()[t.lower()] = pd.read_sql_query(f"""SELECT *
                                  FROM {t}""", con)

In [5]:
# check out attributes for each dataframe
team.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                299 non-null    int64  
 1   team_api_id       299 non-null    int64  
 2   team_fifa_api_id  288 non-null    float64
 3   team_long_name    299 non-null    object 
 4   team_short_name   299 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 11.8+ KB


In [6]:
team_attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              1458 non-null   int64  
 1   team_fifa_api_id                1458 non-null   int64  
 2   team_api_id                     1458 non-null   int64  
 3   date                            1458 non-null   object 
 4   buildUpPlaySpeed                1458 non-null   int64  
 5   buildUpPlaySpeedClass           1458 non-null   object 
 6   buildUpPlayDribbling            489 non-null    float64
 7   buildUpPlayDribblingClass       1458 non-null   object 
 8   buildUpPlayPassing              1458 non-null   int64  
 9   buildUpPlayPassingClass         1458 non-null   object 
 10  buildUpPlayPositioningClass     1458 non-null   object 
 11  chanceCreationPassing           1458 non-null   int64  
 12  chanceCreationPassingClass      14

In [7]:
team[team['team_api_id']==9987]

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN


In [8]:
team.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [9]:
match.columns

Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal',
       ...
       'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA'],
      dtype='object', length=115)

Teams are shown once in the Team table, but multiple times (once per season) in the team_attributes table

In [11]:
team_attributes['year']=None
for i in range(len(team_attributes)):
    team_attributes['year'].iloc[i] = team_attributes['date'].iloc[i][:4]

/Applications/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [12]:
team_attributes = team_attributes[team_attributes['year']=='2015']

In [13]:
teams = pd.merge(team, team_attributes, how='inner', on='team_api_id', )

In [14]:
teams.head()

,id_x,team_api_id,team_fifa_api_id_x,team_long_name,team_short_name,id_y,team_fifa_api_id_y,date,buildUpPlaySpeed,buildUpPlaySpeedClass,...,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,year
0,1,9987,673.0,KRC Genk,GEN,490,673,2015-09-10 00:00:00,58,Balanced,...,Normal,Organised,36,Medium,57,Press,70,Wide,Cover,2015
1,3,10000,15005.0,SV Zulte-Waregem,ZUL,1458,15005,2015-09-10 00:00:00,54,Balanced,...,Little,Organised,44,Medium,58,Press,37,Normal,Cover,2015
2,4,9994,2007.0,Sporting Lokeren,LOK,764,2007,2015-09-10 00:00:00,62,Balanced,...,Normal,Organised,45,Medium,52,Press,52,Normal,Cover,2015
3,5,9984,1750.0,KSV Cercle Brugge,CEB,300,1750,2015-09-10 00:00:00,53,Balanced,...,Normal,Organised,59,Medium,47,Press,62,Normal,Cover,2015
4,6,8635,229.0,RSC Anderlecht,AND,51,229,2015-09-10 00:00:00,52,Balanced,...,Normal,Organised,53,Medium,50,Press,61,Normal,Cover,2015


In [15]:
teams.columns

Index(['id_x', 'team_api_id', 'team_fifa_api_id_x', 'team_long_name',
       'team_short_name', 'id_y', 'team_fifa_api_id_y', 'date',
       'buildUpPlaySpeed', 'buildUpPlaySpeedClass', 'buildUpPlayDribbling',
       'buildUpPlayDribblingClass', 'buildUpPlayPassing',
       'buildUpPlayPassingClass', 'buildUpPlayPositioningClass',
       'chanceCreationPassing', 'chanceCreationPassingClass',
       'chanceCreationCrossing', 'chanceCreationCrossingClass',
       'chanceCreationShooting', 'chanceCreationShootingClass',
       'chanceCreationPositioningClass', 'defencePressure',
       'defencePressureClass', 'defenceAggression', 'defenceAggressionClass',
       'defenceTeamWidth', 'defenceTeamWidthClass', 'defenceDefenderLineClass',
       'year'],
      dtype='object')

In [16]:
teams.drop(columns=['id_x', 'team_fifa_api_id_x', 'team_long_name', 'team_short_name', 'id_y',
                    'team_fifa_api_id_y', 'date', 'year'], inplace=True)

In [17]:
# for this model will only select the numerical columns for attributes
teams.drop(columns=['buildUpPlaySpeedClass', 'buildUpPlayDribblingClass', 'buildUpPlayPassingClass',
                    'buildUpPlayPositioningClass', 'chanceCreationPassingClass', 'chanceCreationCrossingClass',
                    'chanceCreationShootingClass', 'chanceCreationPositioningClass', 'defencePressureClass',
                    'defenceAggressionClass', 'defenceTeamWidthClass', 'defenceDefenderLineClass'], inplace=True)

In [18]:
teams.columns

Index(['team_api_id', 'buildUpPlaySpeed', 'buildUpPlayDribbling',
       'buildUpPlayPassing', 'chanceCreationPassing', 'chanceCreationCrossing',
       'chanceCreationShooting', 'defencePressure', 'defenceAggression',
       'defenceTeamWidth'],
      dtype='object')

In [19]:
list(match.columns)

['id',
 'country_id',
 'league_id',
 'season',
 'stage',
 'date',
 'match_api_id',
 'home_team_api_id',
 'away_team_api_id',
 'home_team_goal',
 'away_team_goal',
 'home_player_X1',
 'home_player_X2',
 'home_player_X3',
 'home_player_X4',
 'home_player_X5',
 'home_player_X6',
 'home_player_X7',
 'home_player_X8',
 'home_player_X9',
 'home_player_X10',
 'home_player_X11',
 'away_player_X1',
 'away_player_X2',
 'away_player_X3',
 'away_player_X4',
 'away_player_X5',
 'away_player_X6',
 'away_player_X7',
 'away_player_X8',
 'away_player_X9',
 'away_player_X10',
 'away_player_X11',
 'home_player_Y1',
 'home_player_Y2',
 'home_player_Y3',
 'home_player_Y4',
 'home_player_Y5',
 'home_player_Y6',
 'home_player_Y7',
 'home_player_Y8',
 'home_player_Y9',
 'home_player_Y10',
 'home_player_Y11',
 'away_player_Y1',
 'away_player_Y2',
 'away_player_Y3',
 'away_player_Y4',
 'away_player_Y5',
 'away_player_Y6',
 'away_player_Y7',
 'away_player_Y8',
 'away_player_Y9',
 'away_player_Y10',
 'away_player

In [20]:
matches = match[['home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']]

In [21]:
matches['result'] = None

for i in range(len(matches)):
    if matches['home_team_goal'].iloc[i] > matches['away_team_goal'].iloc[i]:
        matches['result'].iloc[i] = 'Home Win'
    elif matches['home_team_goal'].iloc[i] < matches['away_team_goal'].iloc[i]:
        matches['result'].iloc[i] = 'Away Win'
    elif matches['home_team_goal'].iloc[i] == matches['away_team_goal'].iloc[i]:
        matches['result'].iloc[i] = 'Tie'

<ipython-input-21-a59616cd06be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['result'] = None
/Applications/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-21-a59616cd06be>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['r

In [22]:
matches.head()

,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,result
0,9987,9993,1,1,Tie
1,10000,9994,0,0,Tie
2,9984,8635,0,3,Away Win
3,9991,9998,5,0,Home Win
4,7947,9985,1,3,Away Win


In [23]:
matches['result'].value_counts(normalize=True)

Home Win    0.458717
Away Win    0.287386
Tie         0.253897
Name: result, dtype: float64

For a naive baseline model, we could always predict a home win and be correct about 46% of the time. The goal is to predict with a better accuracy than that. 

In [24]:
matches_home = pd.merge(matches, teams, how='inner', left_on='home_team_api_id', right_on='team_api_id')

matches_home.head()

,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,result,team_api_id,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
0,9987,9993,1,1,Tie,9987,58,52.0,38,30,69,56,36,57,70
1,9987,9999,1,1,Tie,9987,58,52.0,38,30,69,56,36,57,70
2,9987,9984,3,2,Home Win,9987,58,52.0,38,30,69,56,36,57,70
3,9987,9986,1,0,Home Win,9987,58,52.0,38,30,69,56,36,57,70
4,9987,9998,2,0,Home Win,9987,58,52.0,38,30,69,56,36,57,70


In [25]:
df_matches = pd.merge(matches_home, teams, how='inner', left_on='away_team_api_id', right_on='team_api_id',
                     suffixes=(None, '_away'))

df_matches.head()

,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,result,team_api_id,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,...,team_api_id_away,buildUpPlaySpeed_away,buildUpPlayDribbling_away,buildUpPlayPassing_away,chanceCreationPassing_away,chanceCreationCrossing_away,chanceCreationShooting_away,defencePressure_away,defenceAggression_away,defenceTeamWidth_away
0,9987,9984,3,2,Home Win,9987,58,52.0,38,30,...,9984,53,49.0,43,52,58,42,59,47,62
1,9987,9984,2,0,Home Win,9987,58,52.0,38,30,...,9984,53,49.0,43,52,58,42,59,47,62
2,9987,9984,3,0,Home Win,9987,58,52.0,38,30,...,9984,53,49.0,43,52,58,42,59,47,62
3,9987,9984,4,2,Home Win,9987,58,52.0,38,30,...,9984,53,49.0,43,52,58,42,59,47,62
4,9987,9984,3,3,Tie,9987,58,52.0,38,30,...,9984,53,49.0,43,52,58,42,59,47,62


In [26]:
df_matches.drop(columns=['home_team_goal', 'away_team_goal', 'away_team_api_id', 
                         'team_api_id', 'team_api_id_away'], inplace=True)

In [27]:
df_matches.head()

,home_team_api_id,result,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth,buildUpPlaySpeed_away,buildUpPlayDribbling_away,buildUpPlayPassing_away,chanceCreationPassing_away,chanceCreationCrossing_away,chanceCreationShooting_away,defencePressure_away,defenceAggression_away,defenceTeamWidth_away
0,9987,Home Win,58,52.0,38,30,69,56,36,57,70,53,49.0,43,52,58,42,59,47,62
1,9987,Home Win,58,52.0,38,30,69,56,36,57,70,53,49.0,43,52,58,42,59,47,62
2,9987,Home Win,58,52.0,38,30,69,56,36,57,70,53,49.0,43,52,58,42,59,47,62
3,9987,Home Win,58,52.0,38,30,69,56,36,57,70,53,49.0,43,52,58,42,59,47,62
4,9987,Tie,58,52.0,38,30,69,56,36,57,70,53,49.0,43,52,58,42,59,47,62


In [28]:
df_matches.duplicated().value_counts()

True     11244
False    10197
dtype: int64

### Model building

In [59]:
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score

In [30]:
X = df_matches.drop(columns=['home_team_api_id', 'result'])
y = df_matches['result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [55]:
clf = svm.SVC(probability=True)
clf.fit(X_train, y_train)

SVC(probability=True)

In [56]:
train_preds = clf.predict(X_train)
test_preds = clf.predict(X_test)

In [57]:
train_auc = roc_auc_score(y_train, clf.predict_proba(X_train), multi_class='ovr')
test_auc = roc_auc_score(y_test, clf.predict_proba(X_test), multi_class='ovr')

train_acc = accuracy_score(y_train, train_preds)
test_acc = accuracy_score(y_test, test_preds)

In [58]:
print(f"""Training Scores:
          AUC = {train_auc:.3f}
          Accuracy = {train_acc:.3f}""")
print('-------------------')
print(f"""Testing Scores:
          AUC = {test_auc:.3f}
          Accuracy = {test_acc:.3f}""")

Training Scores:
          AUC = 0.700
          Accuracy = 0.532
-------------------
Testing Scores:
          AUC = 0.602
          Accuracy = 0.502


In [ ]:
param_grid = {}